In [3]:
import sys
import os
import importlib

cur=os.getcwd()
paths = [cur := os.path.dirname(cur) for _ in range(3)]
sys.path.insert(0, paths[0])
from Helpers import loadData, idealParticleGrid, particleGrid, dwOptimizer, posOptimizer
importlib.reload(sys.modules['Helpers'])
importlib.reload(sys.modules['Helpers.visualizer'])

import h5py
import matplotlib.pyplot as plt
import numpy as np
from skimage.feature import peak_local_max

In [24]:
# Establish variable conditions
diameter=100
width=1.3
Cutoff=5
MinSep=5

highHist=250
lowHist=10

max_dw_ticker=10
min_del_diameter=.0001

max_ticker=10
min_del_chi_squared=1

In [25]:
# Load Data
dataFolder = os.path.join(paths[1],'Data')
data = loadData(dataFolder, 'convMap_17.hdf5')
peaks = peak_local_max(data, min_distance=7)


In [29]:
# Extract peak locations
num_peaks = len(peaks)
peak_x_locations = peaks[:, 0]
peak_y_locations = peaks[:, 1]
peak_z_locations = peaks[:, 2]

# Get volume dimensions
grid_width, grid_height, grid_depth = data.shape

In [30]:
# Setup for ideal particle in 3D
kernel_size = 2 * int(diameter/2 + 4*width/2) - 1
half_kernel_size = (kernel_size - 1) / 2
coord_range = np.arange(-half_kernel_size, half_kernel_size + 1)
x_grid, y_grid, z_grid = np.meshgrid(coord_range, coord_range, coord_range, indexing='ij')
radial_grid = np.sqrt(x_grid**2 + y_grid**2 + z_grid**2)

In [31]:
# Create initial grids
peak_grids, overlap_map = particleGrid(
    peak_x_locations - half_kernel_size,
    peak_y_locations - half_kernel_size,
    peak_z_locations - half_kernel_size,
    grid_width, grid_height, grid_depth,
    [1, grid_width, 1, grid_height, 1, grid_depth],
    num_peaks,
    2 * half_kernel_size + 3,
    1  # use_radius=True for initial setup
)

In [ ]:
# Create ideal particle kernels
peak_kernels = idealParticleGrid(peak_grids, diameter, width)

# Calculate initial residuals
residuals = peak_kernels - data
chi_squared = np.sum(residuals**2)
print(f'Initial Chi-Squared={chi_squared}')

# Save initial state
chi_squared_initial = chi_squared
residuals_initial = residuals.copy()

Initial Chi-Squared=3.3052014558359008e+28


In [ ]:
# Optimize Diameter and Width
ticker = 0
del_diameter = 1e99

print("Optimizing diameter and width...")
while (abs(del_diameter) > min_del_diameter) and (ticker < max_dw_ticker):
    del_diameter, del_width = dwOptimizer(peak_grids, residuals, diameter, width)
    diameter = diameter + del_diameter
    width = width + del_width
    
    peak_kernels = idealParticleGrid(peak_grids, diameter, width)
    residuals = peak_kernels - data
    
    print('.', end='', flush=True)
    ticker += 1

print()
chi_squared = np.sum(residuals**2)
print(f'After D/w optimization: Chi-Squared={chi_squared}')

Optimizing diameter and width...
..........
After D/w optimization: Chi-Squared=3.305201455803738e+28


In [ ]:
# Optimize Positions
ticker = 0
del_chi_squared = 1e99

print("Optimizing positions...")
while (abs(del_chi_squared) > min_del_chi_squared) and (ticker < max_ticker):
    
    # We need to store vector components for position optimization
    # Recreate grids with vector information
    x_coords, y_coords, z_coords = np.meshgrid(
        np.arange(1, grid_width + 1),
        np.arange(1, grid_height + 1),
        np.arange(1, grid_depth + 1),
        indexing='ij'
    )
    
    # Create component grids for each particle
    particle_grids_x = np.zeros_like(data)
    particle_grids_y = np.zeros_like(data)
    particle_grids_z = np.zeros_like(data)
    
    for particle_idx in range(num_peaks):
        mask = overlap_map == (particle_idx + 1)
        particle_grids_x[mask] = x_coords[mask] - peak_x_locations[particle_idx]
        particle_grids_y[mask] = y_coords[mask] - peak_y_locations[particle_idx]
        particle_grids_z[mask] = z_coords[mask] - peak_z_locations[particle_idx]
    
    # Calculate radial distances
    particle_grids = np.sqrt(particle_grids_x**2 + particle_grids_y**2 + particle_grids_z**2)
    
    # Optimize positions
    dx_peak_loc, dy_peak_loc, dz_peak_loc = posOptimizer(
        particle_grids, particle_grids_x, particle_grids_y, particle_grids_z,
        overlap_map, residuals, num_peaks, diameter, width
    )
    
    peak_x_locations = peak_x_locations + dx_peak_loc
    peak_y_locations = peak_y_locations + dy_peak_loc
    peak_z_locations = peak_z_locations + dz_peak_loc
    
    # Recreate grids with new positions
    peak_grids, overlap_map = particleGrid(
        peak_x_locations - half_kernel_size,
        peak_y_locations - half_kernel_size,
        peak_z_locations - half_kernel_size,
        grid_width, grid_height, grid_depth,
        [1, grid_width, 1, grid_height, 1, grid_depth],
        num_peaks,
        2 * half_kernel_size + 3,
        1
    )
    
    peak_kernels = idealParticleGrid(peak_grids, diameter, width)
    residuals = peak_kernels - data
    
    new_chi_squared = np.sum(residuals**2)
    del_chi_squared = chi_squared - new_chi_squared
    chi_squared = new_chi_squared
    
    print('.', end='', flush=True)
    ticker += 1

print()
chi_squared = np.sum(residuals**2)
print(f'Final Chi-Squared={chi_squared}')
print(f'Optimized diameter={diameter}, width={width}')

Optimizing positions...
..........
Final Chi-Squared=3.3052014558686464e+28
Optimized diameter=318.6754290566982, width=17.300512443445186
